In [1]:
pip install chess

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pygame
import chess
import random
import chess.polyglot
import torch
import time
import matplotlib.pyplot as plt

pygame 2.5.0 (SDL 2.28.0, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#initialise display
X = 800
Y = 800
scrn = pygame.display.set_mode((X, Y))
pygame.init()
reader = chess.polyglot.open_reader('baron30.bin')
#basic colours
WHITE = (255, 255, 255)
GREY = (128, 128, 128)
YELLOW = (204, 204, 0)
BLUE = (50, 255, 255)
BLACK = (0, 0, 0)


piece_width = 100
piece_height = 100

# Load and resize the piece images
pieces = {
    'p': pygame.transform.scale(pygame.image.load('images/b_p.png').convert(), (piece_width, piece_height)),
    'n': pygame.transform.scale(pygame.image.load('images/b_n.png').convert(), (piece_width, piece_height)),
    'b': pygame.transform.scale(pygame.image.load('images/b_b.png').convert(), (piece_width, piece_height)),
    'r': pygame.transform.scale(pygame.image.load('images/b_r.png').convert(), (piece_width, piece_height)),
    'q': pygame.transform.scale(pygame.image.load('images/b_q.png').convert(), (piece_width, piece_height)),
    'k': pygame.transform.scale(pygame.image.load('images/b_k.png').convert(), (piece_width, piece_height)),
    'P': pygame.transform.scale(pygame.image.load('images/w_p.png').convert(), (piece_width, piece_height)),
    'N': pygame.transform.scale(pygame.image.load('images/w_n.png').convert(), (piece_width, piece_height)),
    'B': pygame.transform.scale(pygame.image.load('images/w_b.png').convert(), (piece_width, piece_height)),
    'R': pygame.transform.scale(pygame.image.load('images/w_r.png').convert(), (piece_width, piece_height)),
    'Q': pygame.transform.scale(pygame.image.load('images/w_q.png').convert(), (piece_width, piece_height)),
    'K': pygame.transform.scale(pygame.image.load('images/w_k.png').convert(), (piece_width, piece_height)),
}

def update(scrn,board):
    '''
    updates the screen basis the board class
    '''
    
    for i in range(64):
        piece = board.piece_at(i)
        if piece == None:
            pass
        else:
            scrn.blit(pieces[str(piece)],((i%8)*100,700-(i//8)*100))
    
    for i in range(7):
        i=i+1
        pygame.draw.line(scrn,WHITE,(0,i*100),(800,i*100))
        pygame.draw.line(scrn,WHITE,(i*100,0),(i*100,800))

    pygame.display.flip()


In [5]:
def main(BOARD):
    scrn.fill(BLACK)
    pygame.display.set_caption('Chess')
    index_moves = []
    status = True
    while (status):
        update(scrn,BOARD)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                status = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                scrn.fill(BLACK)
                pos = pygame.mouse.get_pos()
                square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                index = (7-square[1])*8+(square[0])
                if index in index_moves: 
                    
                    move = moves[index_moves.index(index)]
                    
                    BOARD.push(move)
                    index=None
                    index_moves = []
                else:
                    #check the square that is clicked
                    piece = BOARD.piece_at(index)
                    #if empty pass
                    if piece == None:
                        
                        pass
                    else:
                        all_moves = list(BOARD.legal_moves)
                        moves = []
                        for m in all_moves:
                            if m.from_square == index:                
                                moves.append(m)
                                t = m.to_square
                                TX1 = 100*(t%8)
                                TY1 = 100*(7-t//8)
                                pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                        
                        index_moves = [a.to_square for a in moves]
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

def main_one_agent(BOARD,agent,agent_color):
    scrn.fill(BLACK)
    #name window
    pygame.display.set_caption('Chess')
    index_moves = []
    subtree = [[]]
    status = True
    while (status):
        update(scrn,BOARD)  
        if BOARD.turn==agent_color:
            start_time = time.time()
            tree = [[[BOARD, [0], eval_board(BOARD), 1]]]
            move, layers = agent(tree,subtree)
            BOARD.push(move)
            scrn.fill(BLACK)
            print("time used:", time.time()-start_time, "seconds")
        else:

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    status = False
                if event.type == pygame.MOUSEBUTTONDOWN:
                    scrn.fill(BLACK)
                    pos = pygame.mouse.get_pos()
                    square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                    index = (7-square[1])*8+(square[0])
                    if index in index_moves: 
                        
                        move = moves[index_moves.index(index)]
                        #print(BOARD)
                        #print(move)
                        BOARD.push(move)
                        index=None
                        index_moves = []
                    else:                   
                        piece = BOARD.piece_at(index)                    
                        if piece == None:
                            pass
                        else:

                            all_moves = list(BOARD.legal_moves)
                            moves = []
                            for m in all_moves:
                                if m.from_square == index:          
                                    moves.append(m)
                                    t = m.to_square
                                    TX1 = 100*(t%8)
                                    TY1 = 100*(7-t//8)
                                    pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                            #print(moves)
                            index_moves = [a.to_square for a in moves]
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

In [6]:

def board_to_tensor(board):
    tensor = np.zeros((1,15, 8, 8), dtype=np.float32)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        layer = int(piece_type-1 + 3*(1+color))
        tensor[0,layer, chess.square_rank(square), chess.square_file(square)] = piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            tensor[0,layer, chess.square_rank(s), chess.square_file(s)] += 0.1*color
    turn = (1 if board.turn else -1)
    tensor[0, 12, 0, 0] = (0.5 if board.has_kingside_castling_rights(board.turn) else 0)
    tensor[0, 12, -1, 0] = (-0.5 if board.has_kingside_castling_rights(board.turn==False) else 0)
    tensor[0, 12, 0, -1] = (0.5 if board.has_queenside_castling_rights(board.turn) else 0)
    tensor[0, 12, -1, -1] = (-0.5 if board.has_queenside_castling_rights(board.turn==False) else 0)
    tensor[0, 13, :, :] = turn/640
    if board.is_checkmate():
        tensor[0, 14, :, :] = -turn
    return torch.from_numpy(tensor)
def total_value(Board):
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    total = 0
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        piece_value = values[piece_type-1]
        total += piece_value
    return(total)
def fen_to_tensor(fen):
    board = chess.Board(fen)
    return(board_to_tensor(board))

In [7]:

def eval_board(BOARD):
    #will be replaced by a deep network
    tensor = board_to_tensor(BOARD)[0]
    score = 0
    pieces = BOARD.piece_map()
    if BOARD.is_checkmate() and BOARD.turn == True:
        return(-1000)
    if BOARD.is_checkmate() and BOARD.turn == False:
        return(1000)
    if BOARD.is_stalemate():
        return(0)
    if BOARD.is_insufficient_material():
        return(0)
    if BOARD.has_insufficient_material(BOARD.turn):
        return(-100)
    if BOARD.has_insufficient_material(BOARD.turn == False):
        return(100)
    
    return int(torch.sum(tensor).item()*100)/100
def stable_eval(b):
    value = eval_board(b)
    m = 0
    B = b.copy()
    moves = list(b.legal_moves)
    for move in moves:
        copy = b.copy()
        copy.push(move)
        v = eval_board(copy)
        a = np.abs(v-value)
        if (a>0.9) and (a > m) :
            m = a
            B = copy.copy()
    if m >=0.9:
        return(stable_eval(B))
    else:
        return(value)
            

In [8]:
#initialise chess board

b = chess.Board()


In [9]:
class Parameters:
    def __init__(self, risk, width, depth, agressivity):
        self.risk = risk
        self.width = width
        self.depth = depth
        self.agressivity = agressivity
    

In [10]:
def mini_max(tree, subtree, parameters, eval_board, color):
    #tree[layer,...]
    #layer[nodes,...]
    #initial tree = [[[chess.Board(), [], 0, 1]]]
    #node [board, position, value, fertile]
    #width : matrix : 
    #width[i][j] = number of fertile leafs that we keep after the pruning of layer i+1 using the result of layer i+j+1
    depth = parameters.depth
    lib = []
    moves = list(b.legal_moves)
    if len(moves) ==0:
        return(chess.Move.null(), [[]])
    else :
        opening_move = reader.get(b)
        if opening_move == None:
            pass
        else:
            return (opening_move.move, [[]])
        #sub, id = is_board_in_list(b, tree[-1])
        if False:
            print("recycle")
            layers = subtree(layers, id)
            depth = 4
            width = width[:-4]
        else:
            print("-------------")
            print("initial")
            print("initial board value = " + str(eval_board(b)))
            print(b)
            print("exploration stage")
            for i in range(depth):       
                side = ((i%2 == 0)== color)  #if color == False, i ==0 : side = False
                print(side)
                print("growing")
                old, new = grow(tree[-1], eval_board, side)
                tree[-1] = old
                tree.append(new.copy())
                print("pruning")
                tree, removed = prun(tree, eval_board, parameters, side==False)
                print("sterilizing")
                tree[-1] = sterilize(removed, tree[-1])
                if i>=1:
                    lib.append(tree[-1].copy)
                print("exploration " + str((i+1)*100/depth) + " %")
            print("up propagating")
            best_move, l = up_propagate(tree[:-1], color==(depth%2==0), [])
            print("showing steps")
            print_steps(l)
            return(best_move, tree[2:])
            
            
            

In [11]:
#tree[layer,...]
#layer[nodes,...]
#initial tree = [[[chess.Board(), [0], 0, True]]]
#node [board, position, value, fertile]
#width[i][j] = number of fertile leafs that we keep after the pruning of layer i+1 using the result of layer i+j+1
def grow(leafs, eval_board, side):
    new_layer = []
    for k in range(len(leafs)):
        b = leafs[k][0]
        moves = list(b.legal_moves)
        temp = []
        if len(moves) !=0:
            for i in range(len(moves)):
                copy = b.copy()
                copy.push(moves[i])
                temp.append([copy, leafs[k][1]+ [i], eval_board(copy), 1])
            if side:
                leafs[k][2] = (max(temp, key=lambda x: x[2]))[2]
            else:
                leafs[k][2]= (min(temp, key=lambda x: x[2]))[2]
        new_layer +=temp
    return(leafs, new_layer)
def prun(tree, eval_board, parameters, side):
    tree_depth = len(tree)
    width = parameters.width
    risk = parameters.risk*10**(-(tree_depth-1)/parameters.depth)
    k = 0
    ancestor_depth = tree_depth-3 - 2*k # depth of the common ancestor
    pruned_layer = tree[-2]
    removed = []
    while ancestor_depth >=0:
        print("ancestor_depth = " + str(ancestor_depth))
        pruned_layer, r = prun_layer(pruned_layer, ancestor_depth, width[tree_depth-3][k], risk, side)
        removed +=r
        k+=1
        ancestor_depth = tree_depth-3 - 2*k
    tree[-2] = pruned_layer
    return(tree, removed)

def prun_layer(layer, depth, width, risk, side):
    #depth : depth of the ancestor
    layer = group_with_relatives(layer, depth)
    print("n_group = " + str(len(layer)))
    new_layer = []
    #print(layer[:3])
    print("width  = " + str(width))
    removed = []
    for l in layer:
        l, r = keep_best(l, width, risk, side)
        new_layer+=l  
        removed += r
    return new_layer, removed
def keep_best(L, width, risk, side):
    #print("width  = " + str(width))
    #print("moves = " + str(len(L)))
    if len(L)<= width:
        return L, []
    else:
        L.sort(key=lambda l: l[2], reverse=side)
        w2 = int((1-risk)*width)
        p = L[:w2]
        rest = L[w2:]
        l = len(rest)
        rate = np.exp(np.log(0.05)/l)
        weights = [rate ** i for i in range(l)]
        #for r in rest:
            #r[3] = 0
        selected_indices = random.choices(range(len(rest)), weights, k=width - w2)
        #for idx in selected_indices:
            #rest[idx][3] = 1
        keep = [rest[i] for i in selected_indices]
        removed = [rest[i] for i in range(len(rest)) if i not in selected_indices]
        new = p + keep
        return new, removed
def sterilize(L1, L2):
    print("before pruning : " + str(len(L2)))
    print(sum(item[2] for item in L2) / len(L2))
    l1_dict = {str((l[1])) for l in L1}
    result = [l for l in L2 if (str(l[1][:-1])) not in l1_dict]
    print("after pruning : " + str(len(result)))
    print(sum(item[2] for item in result) / len(result))
    return(result)
def up_propagate(tree, side, l):
    if len(tree)<=2:
        tree[-1].sort(key=lambda l: l[2], reverse=side)
        obj = tree[-1][0]
        l.append([obj])
        b = obj[0].copy()
        return b.pop(), l
    else:
        #L = [l for l in tree[-1] if l[3] == 1]
        L2, r = prun_layer(tree[-1], len(tree)-2, 1, 0, side)
        #L2 = [l for l in L if l[3] == 1]             
        l.append(L2)
        dict_l2 = {str(l2[1][:-1]): l2[2] for l2 in L2}
        for l1 in tree[-2]:
            if str(l1[1]) in dict_l2:
                l1[2] = dict_l2[str(l1[1])]
        return(up_propagate(tree[:-1], side==False, l))
def print_steps(l):
    l.reverse()
    print(l)
    next_ = [0]
    for k in range(len(l)):
        P = True
        for e in l[k]:
            if e[1][:-1]==next_:
                next_ = e[1]
                print(e[0])
                print("value = " + str(e[2]))
                P = False
                continue
        if P:
            print("not found " + str(next_))
def group_with_relatives(L, depth):
    result = {}
    for l in L:
        prefix = l[1][:depth+1]
        if str(prefix) in result:
            result[str(prefix)].append(l)
        else:
            result[str(prefix)] = [l]
    return (list(result.values()))
def unique(L):
    unique_dict = {}
    for sublist in L:
        value = sublist[0].fen()
        if value not in unique_dict:
            unique_dict[value] = sublist
    unique_list = list(unique_dict.values())
    return unique_list

In [12]:
def play_min_maxN(tree, subtree):
    depth = 6
    risk = 0.3
    agressivity = 0.5
    width = [[16, 0, 0, 0, 0],[8, 0, 0, 0, 0],[8, 8, 0, 0, 0],[8, 8, 0, 0, 0],[10, 12, 8, 8, 3],[10, 12, 8, 8, 0],[5, 8, 7, 7, 0]]
    #
    parameters = Parameters(risk, width, depth, agressivity)
    color = False
    print(parameters.depth)
    return mini_max(tree, subtree, parameters, eval_board, color)

In [13]:
L = [[0,[0,1,2]],[0,[0,2,2]],[0,[0,2,7]],[0,[0,3,2]],[0,[0,3,8]]]
print(group_with_relatives(L, 1))

[[[0, [0, 1, 2]]], [[0, [0, 2, 2]], [0, [0, 2, 7]]], [[0, [0, 3, 2]], [0, [0, 3, 8]]]]


In [14]:
b = chess.Board()
#main_one_agent(b,play_min_maxN,False)

In [15]:
boards = [chess.Board()]*1000
start_time = time.time()
for b in boards:
    #a = stable_eval(b)
    a = eval_board(b)
print((time.time()-start_time))

0.4308502674102783


In [17]:
def game_stage(value):
    #100 =>0
    #mid game tipping point : 60
    #end game tipping point : 30
    #king_pawn tipping point : 12
    return(value)
